# Stage 1: Importing dependencies

In [2]:
import numpy as np
import math
import re
import pandas as pd
from bs4 import BeautifulSoup
import random

from google.colab import drive

In [3]:
!pip install bert-for-tf2
!pip install sentencepiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.2/41.2 kB 4.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Created wheel for bert-for-tf2: filename=bert_for_tf2-0.14.9-py3-none-any.whl size=30515 sha256=325d1630fe3781211c8f61182e7f23906939818008051c2ba81dd2d151a28860
  Stored in directory: /root/.cache/pip/wheels/d8/da/50/126d7b8416d9a0e6bf876935c2219a71e72a6529c25e150c56
  Created wheel for params-flow: filename=params_flow-0.8.2-py3-none-any.whl size=19457 sha256=22f8aa1d6d30d0cc8cd72868d0ac16c167c1aa20a56e654f42359ad77d323146
  Stored in directory: /root/.cache/pip/wheels/97/a8/d0/f7419404174976a2686bb98b5c30df01cc71445415f32db9e6
  Created wheel for py-params: filename=py_params-0.10.2-py3-none-any.whl size=7892 sha256=00afc4a1e40035f7d80569c1c8f5dadbdcd6751a10b046c71cac8bfeec76bb28
  Sto

In [4]:
try:
    %tensorflow_version 2.x
except Exception:
    pass
import tensorflow as tf

import tensorflow_hub as hub

from tensorflow.keras import layers
import bert

Colab only includes TensorFlow 2.x; %tensorflow_version has no effect.


# Stage 2: Data preprocessing

## Loading files

We import files from our personal Google drive.

In [5]:
drive.mount("/content/drive")

Mounted at /content/drive


In [6]:
cols = ["sentiment", "id", "date", "query", "user", "text"]
data = pd.read_csv(
    "/content/drive/MyDrive/Colab Notebooks/Learn BERT - most powerful NLP algorithm by Google/training.csv",
    header=None,
    names=cols,
    engine="python",
    encoding="latin1"
)

In [7]:
data.drop(["id", "date", "query", "user"],
          axis=1,
          inplace=True)

In [8]:
data.head(5)

,sentiment,text
0,0,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,is upset that he can't update his Facebook by ...
2,0,@Kenichan I dived many times for the ball. Man...
3,0,my whole body feels itchy and like its on fire
4,0,"@nationwideclass no, it's not behaving at all...."


## Preprocessing

### Cleaning

In [9]:
def clean_tweet(tweet):
    tweet = BeautifulSoup(tweet, "lxml").get_text()
    # Delete the @
    tweet = re.sub(r"@[A-Za-z0-9]+", ' ', tweet)
    # Delete URL links
    tweet = re.sub(r"https?://[A-Za-z0-9./]+", ' ', tweet)
    # Just keep letters and important punctuation
    tweet = re.sub(r"[^a-zA-Z.!?']", ' ', tweet)
    # Remove additional spaces
    tweet = re.sub(r" +", ' ', tweet)
    return tweet

In [10]:
data_clean = [clean_tweet(tweet) for tweet in data.text]

<ipython-input-9-69fbf428a9d8>:2: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  tweet = BeautifulSoup(tweet, "lxml").get_text()


In [11]:
data_labels = data.sentiment.values
data_labels[data_labels == 4] = 1

### Tokenization

We need to create a BERT layer to have access to meta data for the tokenizer (like vocab size).

In [12]:
FullTokenizer = bert.bert_tokenization.FullTokenizer
bert_layer = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/1",
                            trainable=False)
vocab_file = bert_layer.resolved_object.vocab_file.asset_path.numpy()
do_lower_case = bert_layer.resolved_object.do_lower_case.numpy()
tokenizer = FullTokenizer(vocab_file, do_lower_case)

We only use the first sentence for BERT inputs so we add the CLS token at the beginning and the SEP token at the end of each sentence.

In [13]:
def encode_sentence(sent):
    return ["[CLS]"] + tokenizer.tokenize(sent) + ["[SEP]"]

In [14]:
data_inputs = [encode_sentence(sentence) for sentence in data_clean]

### Dataset creation

We need to create the 3 different inputs for each sentence.

In [15]:
def get_ids(tokens):
    return tokenizer.convert_tokens_to_ids(tokens)

def get_mask(tokens):
    return np.char.not_equal(tokens, "[PAD]").astype(int)

def get_segments(tokens):
    seg_ids = []
    current_seg_id = 0
    for tok in tokens:
        seg_ids.append(current_seg_id)
        if tok == "[SEP]":
            current_seg_id = 1-current_seg_id # convert 1 into 0 and vice versa
    return seg_ids

We will create padded batches (so we pad sentences for each batch inpedendently), this way we add the minimum of padding tokens possible. For that, we sort sentences by length, apply padded_batches and then shuffle.

In [16]:
data_with_len = [[sent, data_labels[i], len(sent)]
                 for i, sent in enumerate(data_inputs)]
random.shuffle(data_with_len)
data_with_len.sort(key=lambda x: x[2])
sorted_all = [([get_ids(sent_lab[0]),
                get_mask(sent_lab[0]),
                get_segments(sent_lab[0])],
               sent_lab[1])
              for sent_lab in data_with_len if sent_lab[2] > 7]

In [17]:
# A list is a type of iterator so it can be used as generator for a dataset
all_dataset = tf.data.Dataset.from_generator(lambda: sorted_all,
                                             output_types=(tf.int32, tf.int32))

In [18]:
next(iter(all_dataset))

(<tf.Tensor: shape=(3, 8), dtype=int32, numpy=
 array([[  101, 16183,  8067,  2102,   999,   999,   999,   102],
        [    1,     1,     1,     1,     1,     1,     1,     1],
        [    0,     0,     0,     0,     0,     0,     0,     0]],
       dtype=int32)>,
 <tf.Tensor: shape=(), dtype=int32, numpy=1>)

In [19]:
BATCH_SIZE = 32
all_batched = all_dataset.padded_batch(BATCH_SIZE, padded_shapes=((3, None), ()))

In [ ]:
#all_batched = all_dataset.padded_batch(BATCH_SIZE, padded_shapes=((3, None), ()))

padded_shapes should be a way to indicate how each dimension of our dataset should be treated regarding the padding process. If you have a look at next(iter(all_dataset)) right before apply padded_batch it can give us some clues because it gives us the dimensions/the shape.

In [20]:
next(iter(all_batched))

(<tf.Tensor: shape=(32, 3, 8), dtype=int32, numpy=
 array([[[  101, 16183,  8067,  2102,   999,   999,   999,   102],
         [    1,     1,     1,     1,     1,     1,     1,     1],
         [    0,     0,     0,     0,     0,     0,     0,     0]],
 
        [[  101,  1045,  2123,  1005,  1056,  2113,  2129,   102],
         [    1,     1,     1,     1,     1,     1,     1,     1],
         [    0,     0,     0,     0,     0,     0,     0,     0]],
 
        [[  101,  3753,  2000,  1996,  9726,   999,   999,   102],
         [    1,     1,     1,     1,     1,     1,     1,     1],
         [    0,     0,     0,     0,     0,     0,     0,     0]],
 
        [[  101,  2017,  2024,  2054,  2015,  3308,  1029,   102],
         [    1,     1,     1,     1,     1,     1,     1,     1],
         [    0,     0,     0,     0,     0,     0,     0,     0]],
 
        [[  101,  2651,  2003,  2126,  2488,  2084,  7483,   102],
         [    1,     1,     1,     1,     1,     1,     1,     1],

In [21]:
NB_BATCHES = math.ceil(len(sorted_all) / BATCH_SIZE)
NB_BATCHES_TEST = NB_BATCHES // 10
all_batched.shuffle(NB_BATCHES)
test_dataset = all_batched.take(NB_BATCHES_TEST)
train_dataset = all_batched.skip(NB_BATCHES_TEST)

# Stage 3: Model building

In [22]:
my_sent = ["[CLS]"] + tokenizer.tokenize("Roses are red.") + ["[SEP]"]
bert_layer([tf.expand_dims(tf.cast(get_ids(my_sent), tf.int32), 0),
            tf.expand_dims(tf.cast(get_mask(my_sent), tf.int32), 0),
            tf.expand_dims(tf.cast(get_segments(my_sent), tf.int32), 0)])

[<tf.Tensor: shape=(1, 768), dtype=float32, numpy=
 array([[-9.27935600e-01, -4.10335362e-01, -9.65754986e-01,
          9.07317877e-01,  8.12913775e-01, -1.74174353e-01,
          9.11234498e-01,  3.41952175e-01, -8.74521136e-01,
         -9.99989271e-01, -7.78409779e-01,  9.69385147e-01,
          9.86160576e-01,  6.36962950e-01,  9.48631287e-01,
         -7.51193225e-01, -4.58339483e-01, -7.08104610e-01,
          4.62098330e-01, -6.57927394e-01,  7.60414660e-01,
          9.99994814e-01, -3.96860749e-01,  3.44166070e-01,
          6.16488695e-01,  9.94400144e-01, -7.76633799e-01,
          9.38316584e-01,  9.59452271e-01,  7.32879341e-01,
         -6.93436742e-01,  2.93080479e-01, -9.93785441e-01,
         -1.64551854e-01, -9.67019439e-01, -9.95549619e-01,
          5.32935441e-01, -6.88061118e-01,  1.34714833e-02,
          2.98193898e-02, -9.18356538e-01,  4.20526177e-01,
          9.99988973e-01,  2.52676398e-01,  6.06235325e-01,
         -3.50750268e-01, -1.00000000e+00,  4.975

In [23]:
class DCNNBERTEmbedding(tf.keras.Model):

    def __init__(self,
                 nb_filters=50,
                 FFN_units=512,
                 nb_classes=2,
                 dropout_rate=0.1,
                 name="dcnn"):
        super(DCNNBERTEmbedding, self).__init__(name=name)

        self.bert_layer = hub.KerasLayer(
            "https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/1",  # 24 version Bert large #
            trainable=False)

        self.bigram = layers.Conv1D(filters=nb_filters,
                                    kernel_size=2,
                                    padding="valid",
                                    activation="relu")
        self.trigram = layers.Conv1D(filters=nb_filters,
                                     kernel_size=3,
                                     padding="valid",
                                     activation="relu")
        self.fourgram = layers.Conv1D(filters=nb_filters,
                                      kernel_size=4,
                                      padding="valid",
                                      activation="relu")
        self.pool = layers.GlobalMaxPool1D()

        self.dense_1 = layers.Dense(units=FFN_units, activation="relu")
        self.dropout = layers.Dropout(rate=dropout_rate)
        if nb_classes == 2:
            self.last_dense = layers.Dense(units=1,
                                           activation="sigmoid")
        else:
            self.last_dense = layers.Dense(units=nb_classes,
                                           activation="softmax")

    def embed_with_bert(self, all_tokens):
        _, embs = self.bert_layer([all_tokens[:, 0, :],
                                   all_tokens[:, 1, :],
                                   all_tokens[:, 2, :]])
        return embs

    def call(self, inputs, training):
        x = self.embed_with_bert(inputs)

        x_1 = self.bigram(x) # (batch_size, nb_filters, seq_len-1)
        x_1 = self.pool(x_1) # (batch_size, nb_filters)
        x_2 = self.trigram(x) # (batch_size, nb_filters, seq_len-2)
        x_2 = self.pool(x_2) # (batch_size, nb_filters)
        x_3 = self.fourgram(x) # (batch_size, nb_filters, seq_len-3)
        x_3 = self.pool(x_3) # (batch_size, nb_filters)

        merged = tf.concat([x_1, x_2, x_3], axis=-1) # (batch_size, 3 * nb_filters)
        merged = self.dense_1(merged)
        merged = self.dropout(merged, training)
        output = self.last_dense(merged)

        return output

# Stage 4: Training

In [24]:
NB_FILTERS = 100
FFN_UNITS = 256
NB_CLASSES = 2

DROPOUT_RATE = 0.2

BATCH_SIZE = 32
NB_EPOCHS = 5

In [25]:
Dcnn = DCNNBERTEmbedding(nb_filters=NB_FILTERS,
                         FFN_units=FFN_UNITS,
                         nb_classes=NB_CLASSES,
                         dropout_rate=DROPOUT_RATE)

In [26]:
if NB_CLASSES == 2:
    Dcnn.compile(loss="binary_crossentropy",
                 optimizer="adam",
                 metrics=["accuracy"])
else:
    Dcnn.compile(loss="sparse_categorical_crossentropy",
                 optimizer="adam",
                 metrics=["sparse_categorical_accuracy"])

In [27]:
#checkpoint_path = "./drive/MyDrive/projects/BERT/ckpt_bert_embedding/"
checkpoint_path = "./drive/MyDrive/Colab Notebooks/Learn BERT - most powerful NLP algorithm by Google"

ckpt = tf.train.Checkpoint(Dcnn=Dcnn)

ckpt_manager = tf.train.CheckpointManager(ckpt, checkpoint_path, max_to_keep=1)

if ckpt_manager.latest_checkpoint:
    ckpt.restore(ckpt_manager.latest_checkpoint)
    print("Latest Checkpoint restored!")

In [28]:
class MyCustomCallback(tf.keras.callbacks.Callback):

    def on_epoch_end(self, epoch, logs=None):
        ckpt_manager.save()
        print("Checkpoint saved at {}.".format(checkpoint_path))

In [ ]:
Dcnn.fit(train_dataset,
         epochs=NB_EPOCHS,
         callbacks=[MyCustomCallback()])

Epoch 1/5
  40623/Unknown - 2359s 57ms/step - loss: 0.3977 - accuracy: 0.8214Checkpoint saved at ./drive/MyDrive/Colab Notebooks/Learn BERT - most powerful NLP algorithm by Google.
40623/40623 [==============================] - 2366s 57ms/step - loss: 0.3977 - accuracy: 0.8214
Epoch 2/5
28807/40623 [====================>.........] - ETA: 9:08 - loss: 0.3593 - accuracy: 0.8430

KeyboardInterrupt: ignored

# Stage 5: Evaluation

In [ ]:
results = Dcnn.evaluate(test_dataset)
print(results)

In [ ]:
def get_prediction(sentence):
    tokens = encode_sentence(sentence)

    input_ids = get_ids(tokens)
    input_mask = get_mask(tokens)
    segment_ids = get_segments(tokens)

    inputs = tf.stack(
        [tf.cast(input_ids, dtype=tf.int32),
         tf.cast(input_mask, dtype=tf.int32),
         tf.cast(segment_ids, dtype=tf.int32)],
         axis=0)
    inputs = tf.expand_dims(inputs, 0)

    output = Dcnn(inputs, training=False)

    sentiment = math.floor(output*2)

    if sentiment == 0:
        print("Output of the model: {}\nPredicted sentiment: negative.".format(
            output))
    elif sentiment == 1:
        print("Output of the model: {}\nPredicted sentiment: positive.".format(
            output))

In [ ]:
get_prediction("This movie was pretty interesting.")

In [ ]:
get_prediction("I'd rather not do that again.")